# VQSD Demo
This is a circuit demonstratinga (not yet) scalable implementation of the VQSD algorithm in pennylane.

In [2]:
# imports
import pennylane as qml
from pennylane import numpy as np

In [38]:
# define device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits*2)

In [39]:
n_layers = 2 # num parameter layers
shape = qml.RandomLayers.shape(n_layers=n_layers, n_rotations=2) # properly define the shape of the gate layers

In [40]:
# helper function for generating the operator for expval (since for some reason I can't find a way to do a @ b @ c @ ... in a compact fashion)
def generate_operator(n_qubits):
    if n_qubits == 0:
        return None
    if n_qubits == 1:
        return qml.PauliZ(0)
    return generate_operator(n_qubits - 1) @ qml.PauliZ(n_qubits - 1)

In [41]:
# circuit for purity
@qml.qnode(dev)
def circuit0(weights):
    for i in range(n_qubits*2): # initial state TODO: change this to be able to represent any initial matrix
        qml.Hadamard(i)
        
    # swap operator
    for i in range(n_qubits):
        qml.CNOT(wires=[i, n_qubits + i])
    for i in range(n_qubits):
        qml.CNOT(wires=[n_qubits + i, i])
    for i in range(n_qubits):
        qml.CNOT(wires=[i, n_qubits + i])
        
    return qml.expval(generate_operator(n_qubits))
print(qml.draw(circuit0)(weights))

0: ──H─╭●────╭X────╭●────┤ ╭<Z@Z>
1: ──H─│──╭●─│──╭X─│──╭●─┤ ╰<Z@Z>
2: ──H─╰X─│──╰●─│──╰X─│──┤       
3: ──H────╰X────╰●────╰X─┤       


In [42]:
# circuit for dephase
@qml.qnode(dev)
def circuit1(weights):
    for i in range(n_qubits*2): # initial state TODO: change this to be able to represent any initial matrix
        qml.Hadamard(i)
        # qml.RX(weights[0], wires=i)
        # qml.RY(weights[1], wires=i)
        # qml.RZ(weights[2], wires=i)
    qml.RandomLayers(weights=weights, wires=range(n_qubits)) # should be the alternating one but idk how to implement that
    qml.RandomLayers(weights=weights, wires=range(n_qubits, 2*n_qubits)) # second circuit with the exact same weights as the first
    
    # cost function (DIP test)
    qml.CNOT(wires=[0,1])
    operator = generate_operator(n_qubits)
    probs = qml.probs(wires=range(n_qubits, n_qubits*2)) # trace(dephased(p^2))
    return probs

weights = np.random.random(size=shape)
print(qml.draw(circuit1, expansion_strategy="device")(init_params))

0: ──H─────────────────────╭X─╭X──RZ(0.18)─╭●──RY(0.30)─╭●─┤       
1: ──H──RX(0.21)──RX(0.18)─╰●─╰●───────────╰X───────────╰X─┤       
2: ──H─────────────────────╭X─╭X──RZ(0.18)─╭●──RY(0.30)────┤ ╭Probs
3: ──H──RX(0.21)──RX(0.18)─╰●─╰●───────────╰X──────────────┤ ╰Probs


In [43]:
# cost function
def cost_fn(param):
    purity = circuit0(param)
    probs = circuit1(param)
    dephase = probs[0]
    return purity - probs[0] # tr(p^2) - tr(Z(p^2))

In [44]:
#initialize random weights for the circuit
init_params = np.random.random(size=shape)
print(cost_fn(init_params))

-0.13050167241792113


In [45]:
# optimization loop
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 100
params = init_params

for i in range(steps):
    params = opt.step(cost_fn, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost_fn(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5: -0.2302128
Cost after step    10: -0.3367425
Cost after step    15: -0.4198709
Cost after step    20: -0.4669463
Cost after step    25: -0.4876531
Cost after step    30: -0.4955832
Cost after step    35: -0.4984458
Cost after step    40: -0.4994563
Cost after step    45: -0.4998102
Cost after step    50: -0.4999338
Cost after step    55: -0.4999769
Cost after step    60: -0.4999920
Cost after step    65: -0.4999972
Cost after step    70: -0.4999990
Cost after step    75: -0.4999997
Cost after step    80: -0.4999999
Cost after step    85: -0.5000000
Cost after step    90: -0.5000000
Cost after step    95: -0.5000000
Cost after step   100: -0.5000000
Optimized rotation angles: [[ 1.81824967e-01  1.83404510e-01]
 [ 4.13118286e-05 -1.57068514e+00]]
